In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "3"  # Set the GPUs to use
import torch
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

In [ ]:
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
)

import torch, copy, os
from module.score import evaluation_f1
from module.load_json import *
from module.maps import *
from module.args import print_torch_info
from module.inference import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print_torch_info()

# Paths and Modes

In [ ]:
EVAL_MODE = False

RESULT_ID = 'uncleaned_v22_run_1_ensemble_v3'

ACD_1_CHECKPOINT = 'training_results/uncleaned_v22_run_1/acd_klue_roberta_base_mlm_fine_tuned_uncleaned_v22_run_1/checkpoint-37500'
ACD_2_CHECKPOINT = 'training_results/uncleaned_v22_run_1/acd_monologg_koelectra_base_v3_discriminator_uncleaned_v22_run_1/checkpoint-37500'
ACD_3_CHECKPOINT = 'training_results/uncleaned_v22_run_1/acd_snunlp_kr_electra_discriminator_uncleaned_v22_run_1/checkpoint-37500'

ASC_1_CHECKPOINT = 'training_results/uncleaned_v22_run_1/asc_klue_roberta_base_mlm_fine_tuned_uncleaned_v22_run_1/checkpoint-1608'
ASC_2_CHECKPOINT = 'training_results/uncleaned_v22_run_1/asc_monologg_koelectra_base_v3_discriminator_uncleaned_v22_run_1/checkpoint-1608'
ASC_3_CHECKPOINT = 'training_results/uncleaned_v22_run_1/asc_snunlp_kr_electra_discriminator_uncleaned_v22_run_1/checkpoint-1608'

TEST_DATA_PATH = 'dataset/nikluge-sa-2022-test.jsonl'
EVAL_DATA_PATH = 'dataset/nikluge-sa-2022-dev.jsonl'

In [ ]:
if EVAL_MODE == True:
    TEST_DATA_PATH = EVAL_DATA_PATH
test_data = jsonlload(TEST_DATA_PATH)
print('>>>>> >>>>> >>>>> ', TEST_DATA_PATH, ' <<<<< <<<<< <<<<<', '\n', sep='')

# Load Models and Tokenizers

In [ ]:
acd_model_1 = AutoModelForSequenceClassification.from_pretrained(ACD_1_CHECKPOINT)
acd_model_2 = AutoModelForSequenceClassification.from_pretrained(ACD_2_CHECKPOINT)
acd_model_3 = AutoModelForSequenceClassification.from_pretrained(ACD_3_CHECKPOINT)
acd_tokenizer_1 = AutoTokenizer.from_pretrained(ACD_1_CHECKPOINT)
acd_tokenizer_2 = AutoTokenizer.from_pretrained(ACD_2_CHECKPOINT)
acd_tokenizer_3 = AutoTokenizer.from_pretrained(ACD_3_CHECKPOINT)

asc_model_1 = AutoModelForSequenceClassification.from_pretrained(ASC_1_CHECKPOINT)
asc_model_2 = AutoModelForSequenceClassification.from_pretrained(ASC_2_CHECKPOINT)
asc_model_3 = AutoModelForSequenceClassification.from_pretrained(ASC_3_CHECKPOINT)
asc_tokenizer_1 = AutoTokenizer.from_pretrained(ASC_1_CHECKPOINT)
asc_tokenizer_2 = AutoTokenizer.from_pretrained(ASC_2_CHECKPOINT)
asc_tokenizer_3 = AutoTokenizer.from_pretrained(ASC_3_CHECKPOINT)

In [ ]:
# # v1

# def acd_ensemble(form, acd_pair):
#     acd_model_1.to(device)
#     acd_model_2.to(device)
#     acd_model_3.to(device)
#     acd_model_1.eval()
#     acd_model_2.eval()
#     acd_model_3.eval()

#     acd_encoded_1 = acd_tokenizer_1(form, acd_pair, truncation=True, return_tensors="pt")
#     acd_encoded_1 = {k:v.to(device) for k,v in acd_encoded_1.items()}

#     acd_encoded_2 = acd_tokenizer_2(form, acd_pair, truncation=True, return_tensors="pt")
#     acd_encoded_2 = {k:v.to(device) for k,v in acd_encoded_2.items()}

#     acd_encoded_3 = acd_tokenizer_3(form, acd_pair, truncation=True, return_tensors="pt")
#     acd_encoded_3 = {k:v.to(device) for k,v in acd_encoded_3.items()}

#     with torch.no_grad():
#         acd_outputs_1 = acd_model_1(**acd_encoded_1)['logits']
#         acd_outputs_2 = acd_model_2(**acd_encoded_2)['logits']
#         acd_outputs_3 = acd_model_3(**acd_encoded_3)['logits']

#     acd_outputs = acd_outputs_1 + acd_outputs_2 + acd_outputs_3
#     acd_predictions = acd_outputs.argmax(-1)[0]
    
#     return acd_predictions

In [ ]:
# v2

def acd_ensemble(form, acd_pair):
    acd_model_1.to(device)
    acd_model_2.to(device)
    acd_model_3.to(device)
    acd_model_1.eval()
    acd_model_2.eval()
    acd_model_3.eval()

    acd_encoded_1 = acd_tokenizer_1(form, acd_pair, truncation=True, return_tensors="pt")
    acd_encoded_1 = {k:v.to(device) for k,v in acd_encoded_1.items()}

    acd_encoded_2 = acd_tokenizer_2(form, acd_pair, truncation=True, return_tensors="pt")
    acd_encoded_2 = {k:v.to(device) for k,v in acd_encoded_2.items()}

    acd_encoded_3 = acd_tokenizer_3(form, acd_pair, truncation=True, return_tensors="pt")
    acd_encoded_3 = {k:v.to(device) for k,v in acd_encoded_3.items()}

    with torch.no_grad():
        acd_outputs_1 = acd_model_1(**acd_encoded_1)['logits']
        acd_outputs_2 = acd_model_2(**acd_encoded_2)['logits']
        acd_outputs_3 = acd_model_3(**acd_encoded_3)['logits']

    acd_outputs = acd_outputs_1 + acd_outputs_2 + acd_outputs_3
    
    acd_predictions_1 = acd_outputs_1.argmax(-1).squeeze()
    acd_predictions_2 = acd_outputs_2.argmax(-1).squeeze()
    acd_predictions_3 = acd_outputs_3.argmax(-1).squeeze()
    
    if 0 in [acd_predictions_1, acd_predictions_2, acd_predictions_3]:
        return 0
    
    return 1

In [ ]:
def asc_ensemble(form, asc_pair):
    asc_model_1.to(device)
    asc_model_2.to(device)
    asc_model_3.to(device)
    asc_model_1.eval()
    asc_model_2.eval()
    asc_model_3.eval()

    asc_encoded_1 = asc_tokenizer_1(form, asc_pair, truncation=True, return_tensors="pt")
    asc_encoded_1 = {k:v.to(device) for k,v in asc_encoded_1.items()}

    asc_encoded_2 = asc_tokenizer_2(form, asc_pair, truncation=True, return_tensors="pt")
    asc_encoded_2 = {k:v.to(device) for k,v in asc_encoded_2.items()}

    asc_encoded_3 = asc_tokenizer_3(form, asc_pair, truncation=True, return_tensors="pt")
    asc_encoded_3 = {k:v.to(device) for k,v in asc_encoded_3.items()}

    with torch.no_grad():
        asc_outputs_1 = asc_model_1(**asc_encoded_1)['logits']
        asc_outputs_2 = asc_model_2(**asc_encoded_2)['logits']
        asc_outputs_3 = asc_model_3(**asc_encoded_3)['logits']

    asc_outputs = asc_outputs_1 + asc_outputs_2 + asc_outputs_3
    
    return asc_outputs

# Inference

In [ ]:
for entity_property_pair in entity_property_pairs:
    num_of_ep_pairs = len(entity_property_pair)
    RESULT_SAVE_NAME = f'{RESULT_ID}_num_of_ep_pairs_{num_of_ep_pairs}.json'
    
    pred_data = inference_m_ensemble(acd_ensemble, asc_ensemble, copy.deepcopy(test_data), entity_property_pair)
    
    if EVAL_MODE == False:
        save_path = './'

        jsondump(pred_data, os.path.join(save_path, RESULT_SAVE_NAME))
        pred_data = jsonload(os.path.join(save_path, RESULT_SAVE_NAME))
    
    print(RESULT_SAVE_NAME)    
    print(len(test_data), len(pred_data))
    
    if EVAL_MODE == True:
        print('INFERENCE DATA: ', TEST_DATA_PATH)

        print('EVAL_MODE :', EVAL_MODE)

        result = evaluation_f1(test_data, pred_data)
        print(list(result.items())[0])
        print(list(result.items())[1])
    break